In [ ]:

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy
import skimage

In [ ]:
fname=r"J:\ctgroup\Edward\DATA\VMI\20220613\xe005_e_calibrated.h5"
base_data= pd.read_hdf(fname, key="data")

In [ ]:
data=base_data.query("sqrt(px**2+py**2+pz**2)<0.4").sample(frac=1).reset_index(drop=True)
data=data.query("pz>0")
#
data_sym=data.copy()
data_sym['px']=-data_sym['px']
data_sym['py']=-data_sym['py']
data_sym['pz']=-data_sym['pz']
data=pd.concat([data,data_sym])
del data_sym

In [ ]:

data['pr']=np.sqrt(data['px']**2+data['py']**2+data['pz']**2)
data['phi']=np.arctan2(data['px'],data['pz'])
data['theta']=np.arccos(data['px']/data['pr'])
data['E']=data.pr**2/2
def bump(x):
    return np.where(np.abs(x)<1,np.exp(1/(x**2-1)+1)**0.2,0)

h2d,re,pe=np.histogram2d(data['pr'],data['phi'],bins=1024)
re=1/2*(re[:-1]+re[1:])
pe=1/2*(pe[:-1]+pe[1:])
rr,pp=np.meshgrid(re,pe)

mask=bump((rr-0.25)/0.1)*1/rr
signal=h2d/np.sum(h2d,axis=0,keepdims=True)*mask.T
fig=px.imshow(
        h2d*mask.T, origin='lower',
        x=pe,y=re,
        height=600, width=700,
        aspect='auto',
        color_continuous_scale="Inferno"
)

signal_dog=skimage.filters.difference_of_gaussians(signal, 30, 90, mode='wrap')

px.imshow(
        signal_dog, origin='lower',
        x=pe,y=re,
        height=600, width=700,
        aspect='auto'
).show()

fig.add_trace(go.Contour(
        z=signal_dog, x=pe,y=re,
        contours_coloring='lines',
        line_color='white',
        line_width=2,
        )
)

rel_mins= scipy.signal.argrelmin(signal_dog, axis=0)
p_mins,r_mins= pe[rel_mins[1]], re[rel_mins[0]]
from sklearn.cluster import DBSCAN
labels= DBSCAN(.013).fit(np.vstack((p_mins,r_mins)).T).labels_
fig.add_scatter(x=p_mins, y=r_mins, mode='markers')
fig.show()
px.scatter(x=p_mins, y=r_mins,color=labels).show()

curve_df=pd.DataFrame({"theta": p_mins[labels==2], "r": r_mins[labels==2]})

In [ ]:
curve_df.sort_values("theta", inplace=True)
curve_df['r_smooth']= scipy.interpolate.make_smoothing_spline(curve_df.theta, curve_df.r, lam=.0001)(curve_df.theta)
# curve_df['r_smooth']=curve_df['r']
# curve_df['r_smooth'] = scipy.signal.savgol_filter(curve_df['r'], 51, 3)
curve_df['E']=curve_df.r_smooth**2/2
curve_df['drdt']=np.gradient(curve_df.r_smooth, curve_df.theta)
curve_df['Group Delay']=np.gradient(curve_df.theta, curve_df.E)*2

e_hist,ee=np.histogram(data.pr**2/2,bins=1024,weights=bump((data.pr-0.25)/0.1)*1/data.pr)
ee=1/2*(ee[:-1]+ee[1:])
signal=scipy.interpolate.make_smoothing_spline(ee, e_hist/np.max(e_hist), lam=1e-8)


px.line(x=ee,y=e_hist).add_scatter(x=ee,y=signal(ee)).show()

curve_df['Total Signal']=signal(curve_df.E)

fig=px.line(curve_df, x='theta', y='r_smooth')
fig.add_scatter(x=curve_df.theta, y=curve_df.r, mode='markers', marker=dict(color='red', size=2))
fig.show()
px.scatter(curve_df, x='theta', y='drdt')

px.line(curve_df, x='E', y='theta').show()
px.line(curve_df, x='E', y='Group Delay', range_x=(0.01,0.05),range_y=(0,300)).add_hline(y=110).show()
px.line(curve_df, x='E', y='Total Signal', range_x=(0.01,0.05)).show()





In [ ]:
theory_file=r"D:\DATA\4ranges-volume-momspe-2d-sw.dat"
d=[]
for line in open(theory_file):
    if len(line.strip().split("  "))==6:
        d.append([float(x) for x in line.strip().split('  ')])

d=np.asarray(d)


def fit_fn(theta, theta0, a, b):
    return a * np.cos((theta - theta0) * 2) + b


theory_data=pd.DataFrame({
    'r': d[:,0],
    'theta': d[:,1],
    'I': d[:,2]
})

gate=0.01,0.03
mask=bump((theory_data['r']-0.2)/0.1)
theory_data['I']*=mask

min_theta=[]
min_theta_fit=[]
for r in sorted(theory_data['r'].unique()):
    mask=theory_data['r']==r
    theta=theory_data['theta'][mask].reset_index(drop=True)
    I=theory_data['I'][mask].reset_index(drop=True)
    min_theta.append(theta[np.argmin(I)])
    p0=[0, np.max(I)-np.min(I), np.mean(I)]
    fit=scipy.optimize.curve_fit(fit_fn, theta, I, p0=p0)
    if fit[0][1]<0:
        min_theta_fit.append(fit[0][0])
    else:
        min_theta_fit.append(fit[0][0]+np.pi/2)

rv=sorted(theory_data['r'].unique())
rv=np.asarray(rv)
min_theta=np.array(min_theta)%np.pi
min_theta_fit=np.array(min_theta_fit)%np.pi
# min_theta=np.unwrap(min_theta, np.pi)
# px.line(x=min_theta,y=rv).show()



px.line(x=min_theta,y=rv).add_trace(
        go.Contour(
                x=theory_data['theta'],
                y=theory_data['r'],
                z=theory_data['I'],
                ncontours=30
        )
).add_scatter(
        x=min_theta_fit,
        y=rv,
).show()

theory_min_line=pd.DataFrame({
    'theta': min_theta_fit[(rv>0.15) & (rv<0.23)],
    'r': rv[(rv>0.15) & (rv<0.23)],
})
# theory_min_line["theta_smoothed"]= scipy.interpolate.make_smoothing_spline(theory_min_line.r, theory_min_line.theta, lam=1e-5)(theory_min_line.r)
px.line(theory_min_line, x='theta', y='r').add_scatter(x=theory_min_line.theta, y=theory_min_line.r).show()

theory_min_line['E']=theory_min_line['r']**2/2
theory_min_line['Group Delay']=np.gradient(theory_min_line.theta, theory_min_line.E)*2
px.line(theory_min_line, x='E', y='Group Delay').show()

In [ ]:
fit_data=data.copy()
e_bins=np.linspace(0.015,0.035,num=100)
theta_vec=np.linspace(-np.pi/2,np.pi/2,num=1001)
fit_data["e_index"]=np.searchsorted(e_bins,fit_data['E'])

min_theta_exp=[]

for i in range(100):
    data_sel=fit_data[fit_data['e_index']==i]['phi']
    hist,edges=np.histogram(data_sel,bins=1000)
    t_v=1/2*(edges[:-1]+edges[1:])
    hist=scipy.interpolate.make_smoothing_spline(t_v,hist,lam=0.0001)(t_v)
    # fit=scipy.optimize.curve_fit(fit_fn, t_v,hist)[0]
    min_theta_exp.append(t_v[np.argmin(hist)]%np.pi)
    # px.line(x=t_v,y=fit_fn(t_v,*fit)).add_scatter(x=t_v,y=hist).show()
    #fourier transform

min_theta_exp=np.asarray(min_theta_exp)
min_theta_exp=np.unwrap(min_theta_exp,np.pi)
min_theta_exp= scipy.interpolate.make_smoothing_spline(e_bins,min_theta_exp,lam=1e-8)(e_bins)
px.line(x=e_bins,y=min_theta_exp).show()


px.density_heatmap(data, x='phi', y='E').add_scatter(x=min_theta_exp,y=e_bins).show()

group_delay=np.gradient(min_theta_exp,e_bins)
px.line(x=e_bins,y=group_delay).show()
